Лабораторная работа №4
Импорт и архивирование текста из пдф с веб страницы
Болотов М.В. АСУ4-22-1м

In [1]:
import os                                          # Для работы с файловой системой.
import fitz                                        # Из PyMuPDF для работы с PDF-файлами.
import zipfile                                     # Для создания ZIP-архива.
import requests                                    # Для выполнения HTTP-запросов.
from bs4 import BeautifulSoup                      # Для парсинга веб-страниц.
from urllib.parse import urljoin                   # Объединение относительных URL.

In [ ]:
def clean_filename(filename):                                                          # Очистки имени файла от специальных символов
    return "".join(c if c.isalnum() or c in ('.', '_', '-') else '_' for c in filename)

In [ ]:
def get_pdf_links(url):                                                                # Функция для извлечения PDF-ссылок с заданной веб-страницы
    response = requests.get(url)                                                       # Отправка HTTP-запроса по URL.
    if not response.ok:
        return []                                                                     

    soup = BeautifulSoup(response.text, 'html.parser')                                 # Парсинг HTML-кода страницы
    pdf_links = []                                                                     # Пустой список для хранения PDF-ссылок.
    base_url = 'https://pstu.ru/sveden/education/'                                     # Базовый URL
    for a in soup.find_all('a', href=True):                                            
        link = a['href']                                                               # Получение значения атрибута href
        full_pdf_url = urljoin(base_url, link)                                         # Создание полного URL с помощью urljoin
        if full_pdf_url.endswith('.pdf'):                                              # Проверка, заканчивается ли ссылка на .pdf
            pdf_links.append(full_pdf_url)                                             # Добавление полной PDF-ссылки в список
    return pdf_links                                                                   # Список полных PDF-ссылок

In [ ]:
def extract_text_from_pdf(pdf_url):                                                    # Определение функции для извлечения текста из PDF-файла по его URL
    response = requests.get(pdf_url)                                                   # Отправка HTTP-запроса GET для загрузки PDF.
    if not response.ok:
        return None                                                                    # если запрос не удался

    pdf_data = response.content  
    pdf_document = fitz.open("pdf", pdf_data)                                          # Открытие PDF-документа с использованием PyMuPDF.
    
    text = ""                                                                          # пустая строка для хранения извлеченного текста
    for page_num in range(len(pdf_document)):                                          # Цикл перебор страниц в PDF
        page = pdf_document.load_page(page_num)  
        text += page.get_text()                                                        # Извлечение текста с страницы и добавление в результат.

    return text  

In [ ]:
def download_pdf_text(url, output_folder):                                             # Определение функции для загрузки PDF, извлечения текста и создания ZIP-архива.
    if not os.path.exists(output_folder):                                              # Проверка существования выходной папки.
        os.makedirs(output_folder)                                                     # если отсутствует, создание выходной папки.

    output_zip_filename = os.path.join(output_folder, "pdf_texts.zip")                 # Определение пути к ZIP-файлу

    with zipfile.ZipFile(output_zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:

        visited_urls = set()

        def crawl_and_extract(base_url, url):                                          # Определение рекурсивной функции для обхода и извлечения PDF-файлов.
            visited_urls.add(url)                                                      # Отметка текущего URL как посещенного.
            pdf_links = get_pdf_links(url)                                             # Получение полных PDF-ссылок с текущего URL
            
            for pdf_link in pdf_links:                                                 # Перебор PDF-ссылок и извлечение текста
                if pdf_link not in visited_urls:                                       # Проверка, не посещена ли PDF-ссылка
                    text = extract_text_from_pdf(pdf_link)                             # Извлечение текста из PDF.
                    if text:                                                           # Если текст был успешно извлечен.
                        pdf_filename = os.path.basename(pdf_link)                      # Получение имени PDF-файла.
                        txt_filename = pdf_filename.replace('.pdf', '.txt')            # Генерация имени соответствующего TXT-файла
                        zipf.writestr(txt_filename, text.encode('utf-8'))              # Запись текста в ZIP-файл.
                    visited_urls.add(pdf_link)                                         # Отметка PDF-ссылки как посещенной.
            
            for link in get_pdf_links(url):                                            # Рекурсивное обход дочерних ссылок, найденных на текущей странице
                if link not in visited_urls:                                           # Проверка, не посещена ли дочерняя ссылка.
                    full_child_url = urljoin(base_url, link)                           # Создание полного URL дочерней ссылки
                    crawl_and_extract(base_url, full_child_url)                        # Рекурсивный вызов функции для дочерней ссылки.

        crawl_and_extract(url, url)                                                    # Начало обхода и извлечения с указанного URL.

    return output_zip_filename                                                         # Возврат пути к ZIP-архиву.

In [ ]:
def main():                                                                            # Определение главной функции
    url = "https://pstu.ru/sveden/education/"                                          # Указание URL для начала обхода.
    output_folder = r"C:\Users\79125\OneDrive\Рабочий стол\Учёба ПНИПУ АСУ\3 семестр\Интеллектуальный анализ Web данных\пдф"   # Обозначение папки сохранения
    zip_filename = download_pdf_text(url, output_folder)                               # Вызов функции загрузки PDF и извлечения текста
    if zip_filename:
        print(f"Тексты из PDF успешно загружены и сохранены в: {zip_filename}")

if __name__ == "__main__":
    main()  